In [ ]:
queries = [
    """
    INSERT INTO page_lookup_nonredirect 
            SELECT  page.page_id as redirect_id, page.page_title as redirect_title, page.page_title true_title, 
                    page.page_id, page.page_latest 
            FROM page LEFT OUTER JOIN redirect ON page.page_id = redirect.rd_from
            WHERE redirect.rd_from IS NULL
    """,
    """
    insert into page_lookup_redirect 
            select original_page.page_id redirect_id, original_page.page_title redirect_title, 
                    final_page.page_title as true_title, final_page.page_id, final_page.page_latest 
            from page final_page join redirect on (redirect.page_title = final_page.page_title) 
                join page original_page on (redirect.rd_from = original_page.page_id)
    """,
    """
    INSERT INTO page_lookup
            SELECT redirect_id, redirect_title, true_title, page_id, page_version
            FROM (
                SELECT redirect_id, redirect_title, true_title, page_id, page_version
                FROM page_lookup_nonredirect
                UNION ALL
                SELECT redirect_id, redirect_title, true_title, page_id, page_version
                FROM page_lookup_redirect) u
    """,
    """
           INSERT INTO filtered_pagecounts 
           SELECT regexp_replace (reflect ('java.net.URLDecoder','decode', reflect ('java.net.URLDecoder','decode',pvs.page_title)),'^\s*([a-zA-Z0-9]+).*','$1') page_title 
                ,SUM (pvs.views) AS total_views, SUM (pvs.bytes_sent) AS total_bytes_sent
            FROM pagecounts as pvs 
           WHERE not pvs.page_title LIKE '(MEDIA|SPECIAL||Talk|User|User_talk|Project|Project_talk|File|File_talk|MediaWiki|MediaWiki_talk|Template|Template_talk|Help|Help_talk|Category|Category_talk|Portal|Wikipedia|Wikipedia_talk|upload|Special)\:(.*)' and
                pvs.page_title LIKE '^([A-Z])(.*)' and
                not pvs.page_title LIKE '(.*).(jpg|gif|png|JPG|GIF|PNG|txt|ico)$' and
                pvs.page_title <> '404_error/' and 
                pvs.page_title <> 'Main_Page' and 
                pvs.page_title <> 'Hypertext_Transfer_Protocol' and 
                pvs.page_title <> 'Favicon.ico' and 
                pvs.page_title <> 'Search' and 
                pvs.dt = '2020-01-01'
          GROUP BY 
                regexp_replace (reflect ('java.net.URLDecoder','decode', reflect ('java.net.URLDecoder','decode',pvs.page_title)),'^\s*([a-zA-Z0-9]+).*','$1')
    """,
    """
    INSERT INTO normalized_pagecounts
           SELECT pl.page_id page_id, REGEXP_REPLACE(pl.true_title, '_', ' ') page_title, pl.true_title page_url, views, bytes_sent
           FROM page_lookup pl JOIN filtered_pagecounts fp 
           ON fp.page_title = pl.redirect_title where fp.dt='2020-01-01'
    """
]

In [ ]:
# Scan the catalog from a file. Note that this is for demo only. 
# Use dbcat (https://github.com/tokern/dbcat) to load the catalog from a database
# or a data warehouse


from dbcat.scanners.json import File
catalog_objects = File("test", "test/catalog.json").scan()

In [ ]:
# Load the catalog. For more details, refer to https://github.com/tokern/dbcat
# Provide credentials of the postgres database where catalog is stored.
# Note that you should have already created the database and user 

from data_lineage import catalog_connection

catalog_conf = """
catalog:
  type: postgres
  user: catalog_user
  password: catal0g_passw0rd
  host: 127.0.0.1
  port: 5432
  database: tokern
"""
catalog = catalog_connection(catalog_conf)
catalog.save_catalog(catalog_objects)

In [ ]:
from data_lineage.parser import parse_queries, visit_dml_queries

# Parse all queries
parsed = parse_queries(queries)

# Visit the parse trees to extract source and target queries
visited = visit_dml_queries(catalog, parsed)

In [ ]:
# Create a graph and visualize it

from data_lineage.parser import create_graph
graph = create_graph(catalog, visited)

import plotly
plotly.offline.iplot(graph.fig())

In [ ]:
# Visualize a sub graph 

target = catalog.get_column(database_name="test",
                           schema_name="default",
                           table_name="normalized_pagecounts",
                           column_name="page_title")

sub_graph = graph.sub_graph(target)
sub_fig = sub_graph.fig()
plotly.offline.iplot(sub_fig)